# Create an interface between two materials with minimal strain

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials using the Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials
2. Set Input Parameters (e.g. `MILLER_INDICES`, `THICKNESS`, `MAX_AREA`) below or use the default values
3. Click "Run" > "Run All Cells" to run all cells
4. Wait for the run to complete (depending on the area, it can take 1-2 min or more). Scroll down to view cell results.
5. Review the strain plot and modify its parameters as needed

## Methodology

The following happens in the script below:

1. Create slabs for each input material. The materials data is passed in from and back to the web application according to this description (TBA).
   We assume that two input materials are either in bulk form (e.g. Ni crystal) or layered (e.g. graphene). 
   
   We construct the interface along the Z-axis. The material corresponding to the bottom of the interface is referred to as the "**substrate**", and the top - as the "**layer**". 

2. Perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered, that can be configured by editing the cells below.

3. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. We create the corresponding supercells and place them at a specified distance from each other (note no shift is performed currently).


## 1. Set Input Parameters

### 1.1. Select Substrate and Layer from Input Materials

In [ ]:
SUBSTRATE_PARAMETERS = {
    "MILLER_INDICES": (1, 1, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 3,  # in layers
}

LAYER_PARAMETERS = {
    "MILLER_INDICES": (0, 0, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 1,  # in layers
}

USE_CONVENTIONAL_CELL = True  # if True, the surface plane is constructed using miller indices of the conventional cell

### 1.2. Set Interface Parameters

The distance between layer and substrate and maximum area to consider when matching.


In [ ]:
INTERFACE_PARAMETERS = {
    "DISTANCE_Z": 3.0,  # in Angstroms
    "MAX_AREA": 400,  # in Angstroms^2
}

### 1.3. Set Algorithm Parameters

In [ ]:
ZSL_PARAMETERS = {
    "MAX_AREA": INTERFACE_PARAMETERS["MAX_AREA"],  # The area to consider in Angstrom^2
    "MAX_AREA_TOL": 0.09,  # The area within this tolerance is considered equal
    "MAX_LENGTH_TOL": 0.03,  # supercell lattice vectors lengths within this tolerance are considered equal
    "MAX_ANGLE_TOL": 0.01,  # supercell lattice angles within this tolerance are considered equal
    "STRAIN_TOL": 10e-6,  # strains within this tolerance are considered equal
}

## 2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` as directed in README.

In [ ]:
import sys
if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb","../../config.yml")

## 3. Load input Materials


In [ ]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

from utils.visualize import visualize_materials as visualize

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
visualize(materials, repetitions=[1,1,1], rotation="0x")

## 4. Create interfaces

### 4.1. Extract Interfaces and Terminations

Extract all possible layer/substrate supercell combinations within the maximum area including different terminations.

In [ ]:
from mat3ra.made.tools.build import create_interfaces

interface_data_holder = create_interfaces(
    substrate=materials[0],
    layer=materials[1],
    settings={
        "SUBSTRATE_PARAMETERS": SUBSTRATE_PARAMETERS,
        "LAYER_PARAMETERS": LAYER_PARAMETERS,
        "USE_CONVENTIONAL_CELL": USE_CONVENTIONAL_CELL,
        "ZSL_PARAMETERS": ZSL_PARAMETERS,
        "INTERFACE_PARAMETERS": INTERFACE_PARAMETERS,
    },
    sort_by_strain_and_size=True,
    remove_duplicates=True,
)

### 4.2. Print out the interfaces and terminations

In [ ]:
print("Found terminations:", interface_data_holder.terminations)
print(f"Number of interfaces for a termination:  {len(interface_data_holder.get_interfaces_for_termination(0))}")

### 4.2. Print out interfaces with the lowest strain for each termination

In [ ]:
for termination in interface_data_holder.terminations:
    print(f"Interface with lowest strain for termination {termination} (index 0):")
    interfaces = interface_data_holder.get_interfaces_for_termination(termination)
    first_interface = interfaces[0]
    print(f"    strain: {first_interface.get_mean_abs_strain() * 100:.3f}%")
    print("    number of atoms:", first_interface.num_sites)

## 5. Plot the results

Plot the number of atoms vs strain. Adjust the parameters as needed.


In [ ]:
from utils.plot import plot_strain_vs_atoms

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}

plot_strain_vs_atoms(interface_data_holder, PLOT_SETTINGS)

print("Terminations: \n", interface_data_holder.terminations)

## 6. Select the interface to pass outside this kernel

### 6.1. Select the interface with the desired termination and strain

Select the index for termination first, and for it - the index in the list of corresponding interfaces sorted by strain (index 0 has minimum strain).

In [ ]:
# Could be either the termination as tuple, e.g. `('Ni_P6/mmm_1', 'C_C2/m_2')` or its index: `0`
termination_or_its_index = 0
# select the first interface with the lowest strain and the smallest number of atoms
interfaces_slice_range_or_index = slice(0,1)
selected_interfaces = interface_data_holder.get_interfaces_as_materials(termination_or_its_index, interfaces_slice_range_or_index)

### 6.2. Visualize the selected interface(s)

In [ ]:
visualize(selected_interfaces, repetitions=[1,1,1], rotation="0x")

### 6.3. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import set_data
set_data("materials", selected_interfaces)